## Invoice PDF Stamping Template

### pdf data extraction

In [ ]:
import pdfplumber
import re
from datetime import datetime
import json

pdf_path = "../Foresango USD Sample Invoice.pdf"
with pdfplumber.open(pdf_path) as pdf:
    text = "\n".join(page.extract_text() or '' for page in pdf.pages)
    print(text)

def extract_data(pdf_text):
    # Identify document type
    document_type = None
    if re.search(r'(?i)(Fiscal Tax Invoice)', pdf_text):
        document_type = "FISCALINVOICE"
    elif re.search(r'(?i)(Credit Note)', pdf_text):
        document_type = "CREDITNOTE"
    elif re.search(r'(?i)(Debit Note)', pdf_text):
        document_type = "DEBITNOTE"

    # Extract Document No.
    document_no = re.search(r'Document No\.\s*(\S+)', pdf_text)
    document_no = document_no.group(1) if document_no else None

    # Extract Date
    date_match = re.search(r'Date\s+(\d{2}/\d{2}/\d{4})', pdf_text)
    formatted_date = None
    if date_match:
        receipt_date = date_match.group(1)  # Extract the captured group
        # Combine extracted date with current time
        extracted_date = datetime.strptime(receipt_date, '%d/%m/%Y')
        current_time = datetime.now().time()
        combined_datetime = datetime.combine(extracted_date.date(), current_time)
        formatted_date = combined_datetime.strftime('%Y-%m-%dT%H:%M:%S')

    # Extract currency
    currency = None
    if re.search(r'\bUSD\b', pdf_text):
        currency = "USD"
    elif re.search(r'\bZWG\b', pdf_text):
        currency = "ZWG"

    # Split the text based on item lines (after the headers, which usually appear after 'Excl VAT')
    item_lines = re.split(r'\n(?=\w)', pdf_text)  # Split at newlines with words, avoiding header lines
    
    # Extract item details from each line
    items = []
    total_payment = 0  # Initialize total payment
    for line in item_lines:
        # Clean the line by removing unwanted text
        line = line.strip()
        # Skip the header line with column names and any other irrelevant lines
        if re.match(r'^(Description|Qty|Unit Price|Excl Total VAT)', line):
            continue

        # Match individual item lines and capture the necessary details (description, qty, unit price, VAT)
        match = re.match(r'([a-zA-Z\s0-9]+)\s+(\d+)\s+(\d+\.\d{2})\s+(\d+\.\d{2})\s+(\d+%)', line)
        if match:
            # Convert VAT percentage to float by removing % and dividing by 100
            vat_percent = float(match[5].replace('%', ''))
            # Round unit price to 2 decimal places
            unit_price = round(float(match[3]), 2)
            quantity = int(match[2])

            # Add the item to the list
            items.append({
                "item_name": match[1].strip(),  # Now correctly captures the description
                "tax_percent": round(vat_percent, 2),  # VAT percentage as a float
                "quantity": quantity,  # Quantity is now correctly extracted as an integer
                "unit_price": unit_price,  # Unit Price rounded to 2 decimal places
            })

            # Calculate total payment (quantity * unit price)
            total_payment += quantity * unit_price

    return {
        "deviceID": 19250,
        "receiptType": document_type,
        "receiptCurrency": currency,
        "receiptCounter": 2,  # Placeholder for dynamic value
        "receiptGlobalNo": 2,  # Placeholder for dynamic value
        "invoiceNo": document_no,
        "receiptDate": datetime.now().strftime('%Y-%m-%dT%H:%M:%S'),
        "receiptLines": items,
        "receiptPayments": [{"moneyTypeCode": 1, "paymentAmount": round(total_payment, 2)}]  # Include total payment
    }


# Call the function and print the result
extracted_invoice = extract_data(text)

print(json.dumps(extracted_invoice, indent=4))


{
    "deviceID": 19250,
    "receiptType": "FISCALINVOICE",
    "receiptCurrency": "USD",
    "receiptCounter": 2,
    "receiptGlobalNo": 2,
    "invoiceNo": "FUR57",
    "receiptDate": "2024-12-11T12:15:45",
    "receiptLines": [
        {
            "item_name": "Coke Can",
            "tax_percent": 15.0,
            "quantity": 2,
            "unit_price": 1.5
        },
        {
            "item_name": "Fanta 500Ml",
            "tax_percent": 0.0,
            "quantity": 3,
            "unit_price": 2.0
        },
        {
            "item_name": "Sprite",
            "tax_percent": 15.0,
            "quantity": 2,
            "unit_price": 1.0
        }
    ],
    "receiptPayments": [
        {
            "moneyTypeCode": 1,
            "paymentAmount": 11.0
        }
    ]
}


### Fiscalising the Invoice

In [9]:
import requests


url = "https://nexus-backend.up.railway.app/submitReceipt"
body = {
    "deviceID": "19250",
    "tinNo": "2000168681",
    "receipt": extracted_invoice
}

response = requests.post(url, json=body)

print(json.dumps(response.json(), indent=4))

{
    "qrCode": "https://fdmstest.zimra.co.zw/00000192501112202400000000025f9fd32175f1a5e0"
}


### Stamping the QR Code onto the Invoice

In [12]:
import qrcode
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from PyPDF2 import PdfReader, PdfWriter

# Generate QR code image
qrurl = response.json()['qrCode']
qr = qrcode.QRCode(
    version=1,
    error_correction=qrcode.constants.ERROR_CORRECT_L,
    box_size=10,
    border=4,
)
qr.add_data(qrurl)
qr.make(fit=True)

qr_img = qr.make_image(fill_color="black", back_color="white")
qr_img.save("qr.png")

# Create a temporary PDF with the QR code
qr_pdf_path = "qr_code.pdf"
c = canvas.Canvas(qr_pdf_path, pagesize=letter)

# Get the width and height of the page
width, height = letter

# Set the QR code size (adjust if necessary)
qr_size = 100  # Adjust the size of the QR code

# Position QR code at the bottom-right corner
x_position = width - qr_size - 20  # 20 pixels padding from the right edge
y_position = 20  # 20 pixels padding from the bottom edge

# Draw the QR code on the page
c.drawImage("qr.png", x_position, y_position, width=qr_size, height=qr_size)
c.save()

# Now, merge the QR code PDF with the original PDF
pdf_path = "../Foresango USD Sample Invoice.pdf"
output_path = "../Foresango USD Sample Invoice with QR.pdf"

# Read the original PDF and QR code PDF
reader = PdfReader(pdf_path)
qr_reader = PdfReader(qr_pdf_path)

# Create a writer object to save the output PDF
writer = PdfWriter()

# Get the first page of the original PDF
page = reader.pages[0]

# Merge the QR code page onto the original page
page.merge_page(qr_reader.pages[0])

# Add the merged page to the writer
writer.add_page(page)

# Write the output PDF
with open(output_path, "wb") as output_file:
    writer.write(output_file)

print("QR code added to PDF at the bottom-right corner")


QR code added to PDF at the bottom-right corner
